Goal: Download DESE attendance indicators for all schools from 2010 to 2015 and add to a master dataframe. 

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

The DESE attendance report has been updated in the last six months and doesn't show a full link with arguments that can be changed when the export button is pressed. Fortunately, the same attendance indicators are present on the individual profile pages for schools, so we looped through all school pages between 2010 and 2015 to collect this data. 

First, we pulled all school codes from 2010 to 2015 from the master student race/ethnicity file we created. 

In [6]:
org_codes = pd.read_csv("student_raceeth_all.csv", dtype={"Org Code": object}, usecols = ["SCHOOL", "Org Code", "Year"])

The function below creates a list of org codes for schools that are open in a particular year. Each code is then added to the school profile URL that contains attendance indicators. Attendance indicators are extracted from the html (indicators are slightly different between 2010-2012 and 2013-2015), and then transposed to show indicators in columns instead of rows. 

In [7]:
## This works

def get_school_attendance_indicators(year):
    mask = org_codes["Year"] == year
    code_list = org_codes[mask]
    code_list = list(set(code_list['Org Code'].tolist()))
    
    dfs = []
    for code in code_list:
        year = str(year)
        attendance = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode={}&orgtypecode=6&leftNavId=303&fycode={}'
        attendance = attendance.format(code, year)
        attendance = pd.read_html(attendance)
        attendance = attendance[1][1]
        attendance[0] = code
        if year in ['2013', '2014', '2015']:
            attendance[7] = year
        else:
            attendance[9] = year
        attend_df = pd.DataFrame(attendance)
        attend_df = attend_df.transpose()
        dfs.append(attend_df)
    return dfs

I ran the function separately for each year because I ran into some timeouts. Since 2013-2015 profiles contain the same indicators, I created the first master data frame for these three years:

In [ ]:
attendance_2015 = get_school_attendance_indicators(2015)
attendance_2014 = get_school_attendance_indicators(2014)
attendance_2013 = get_school_attendance_indicators(2013)

In [282]:
attend_13_15 = attendance_2015 + attendance_2014 + attendance_2013
attend_df_13_15 = pd.concat(attend_13_15)

I used one school's profile to pull column headers and write the first 3 years' of data to csv

In [284]:
sch = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode=00100055&orgtypecode=6&leftNavId=303&fycode=2015'
headers = pd.read_html(sch)
headers = list(headers[1][0])
headers[0] = "Org Code"
headers_13_15 = headers + ["Year"]

In [292]:
attend_df_13_15.columns = headers_13_15
attend_df_13_15 = attend_df_13_15.reset_index()
attend_df_13_15 = attend_df_13_15.set_index('Org Code')
attend_df_13_15.to_csv("student_attendance_2013_2015.csv")

We repeated the process for years 2010-2012, using a 2012 profile to pull the correct column headers

In [13]:
attendance_2010 = get_school_attendance_indicators(2010)
attendance_2011 = get_school_attendance_indicators(2011)
attendance_2012 = get_school_attendance_indicators(2012)

In [20]:
sch = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode=00100055&orgtypecode=6&leftNavId=303&fycode=2012'
headers = pd.read_html(sch)
headers = list(headers[1][0])
headers[0] = "Org Code"
headers_10_12 = headers + ["Year"]

In [21]:
attend_10_12 = attendance_2012 + attendance_2011 + attendance_2010
attend_df_10_12 = pd.concat(attend_10_12)
attend_df_10_12.columns = headers_10_12

attend_df_10_12

,Org Code,Attendance Rate,Average # of days absent,Absent 10 or more days,Chronically Absent (10% or more),In-School Suspension Rate,Out-of-School Suspension Rate,Unexcused Absences > 9,Retention Rate,Year
1,01140305,94.1,10.0,37.7,16.8,5.9,8.7,0.0,0.4,2012
1,02810190,94.1,9.6,36.9,20.5,0.3,11.3,8.6,0.9,2012
1,00160050,96.1,6.7,22.6,10.0,0.0,0.0,0.0,0.5,2012
1,06580030,96.7,5.8,17.7,4.6,0.0,0.0,0.0,0.0,2012
1,04430205,95.3,9.0,37.6,11.6,0.0,22.4,20.8,0.0,2012
1,01850075,97.1,5.0,12.6,4.4,0.0,0.0,0.0,0.0,2012
1,07250010,96.6,5.7,14.3,5.4,0.0,0.3,0.0,0.0,2012
1,00350266,95.1,8.7,32.7,10.2,0.0,1.2,17.6,1.9,2012
1,02070100,96.7,5.8,18.1,2.1,0.0,0.0,0.0,0.0,2012
1,06800505,95.4,8.1,28.0,9.3,0.0,7.5,0.0,2.0,2012


We kept only attendance indicators shared between 2010-2012 and 2013-2015. Once we had a master dataframe, we exported to csv. 

In [ ]:
attend_df_10_12_cut = attend_df_10_12[['Org Code', 'Attendance Rate', "Average # of days absent", "Absent 10 or more days", "Chronically Absent (10% or more)", "Year", "Retention Rate"]]
attend_df_13_15 = attend_df_13_15.reset_index()
attend_df_13_15_cut = attend_df_13_15[['Org Code', 'Attendance Rate', "Average # of days absent", "Absent 10 or more days", "Chronically Absent (10% or more)", "Year", "Retention Rate"]]

In [384]:
attendance_all = pd.concat([attend_df_10_12_cut, attend_df_13_15_cut])

In [385]:
attendance_all
attendance_all.reset_index()
attendance_all = attendance_all.set_index('Org Code')
attendance_all.to_csv("student_attendance_all.csv")